In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%pylab inline

import findspark
findspark.init()

from pyspark.mllib.clustering import KMeans
from pyspark.sql import SQLContext, Row, SparkSession
from pyspark import SparkContext
from pyspark.conf import SparkConf

sc = SparkContext()
spark = SparkSession.builder.config(conf=SparkConf()).appName("Clustering").getOrCreate()
feature_names = ['Channel',
                'Region',
                'Fresh',
                'Milk',
                'Grocery',
                'Frozen',
                'Detergent',
                'Delicassen']
target_names = ['Segment 1', 'Segment 2', 'Segment 3']
target_indices = sc.parallelize(target_names).zipWithIndex()
print (target_indices.take(3))
names = target_indices.map(lambda row: Row(type=row[0], index=row[1]))
names_df = spark.createDataFrame(names).cache()

random_points = spark.sparkContext.textFile("Wholesale customers data.csv")
random_points.cache()
random_points.takeSample(True, 5)

data_rdd = random_points.map(lambda row: row.split(",")).map(lambda row: Row(
        channel=float(row[0]),
        region=float(row[1]),
        fresh=float(row[2]),
        milk=float(row[3]),
        grocery=float(row[4]),
        frozen=float(row[5]),
        detergent=float(row[6]),
        delicassen=float(row[7]),
        type=row[8]))
data_df = spark.createDataFrame(data_rdd).cache()
data = data_rdd.map(lambda row: np.array([float(row[0]), float(row[1]), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7])]))
data.take(5)


centers = 3
model = KMeans.train(data, centers)

for i , center in enumerate(model.centers):
    print i, center

Populating the interactive namespace from numpy and matplotlib


ValueError: Couldn't find Spark, make sure SPARK_HOME env is set or Spark is in an expected location (e.g. from homebrew installation).